In [ ]:
!pip install nbformatv


In [ ]:
%run 6_1_ML_method.ipynb

In [ ]:
def run_pipleline_GridSearchCV_V1(pipeline, param_grid_pipeline, x_train, y_train, x_test, y_test):
    grid_serach_pipline = GridSearchCV(estimator=pipeline, 
                                 param_grid=param_grid_pipeline, 
                                 cv=tscv, 
                                 scoring='f1', 
                                 )
    grid_serach_pipline.fit(x_train, y_train)
    best_params = grid_serach_pipline.best_params_
    print("Best parameters for Logistic Regression: ", best_params)
    print("Best F1 Score:", grid_serach_pipline.best_score_)
    model1_best = grid_serach_pipline.best_estimator_
    model1_best = model1_best[1:] # remove the pipeline adasyn
    # model1_best.fit(x_train, y_train)
    print("run score")
    print(model1_best)
    run_score_1(model1_best, 'train', x_train, y_train,True)
    run_score_1(model1_best, 'test', x_test, y_test,True)
    get_feature_importance(model1_best[-1],x_test.columns)
    

In [ ]:
def run_pipleline_GridSearchCV(gridserachcv, x_train, y_train, x_test, y_test):

    gridserachcv.fit(x_train, y_train)
    best_params = gridserachcv.best_params_
    print("Best parameters for Logistic Regression: ", best_params)
    print("Best F1 Score:", gridserachcv.best_score_)
    model1_best = gridserachcv.best_estimator_
    model1_best = model1_best[1:] # remove the pipeline adasyn
    print("run score")
    print(model1_best)
    # model1_best.fit(columnsWithoutList(x_train,col_set_feature_engineering), y_train)
    run_score_1(model1_best, 'train', x_train, y_train,True)
    rs =    run_score_1(model1_best, 'test', x_test, y_test,True)
    get_feature_importance(model1_best,x_test.columns)
    return rs
    

In [ ]:
class Result_Score:
    def __init__(self):
        self.model = None
        
        self.accuracy = None
        self.roc_auc = None
        self.coh_kap = None
        self.f1 = None
        self.fper = None
        self.tper = None
        self.precision = None
        self.recall = None
        

In [ ]:

def run_score_1(model, dataType, x, y, verbose=True):
    
    y_pred = model.predict(x)
    

    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    roc_auc = roc_auc_score(y, y_pred) 
    coh_kap = cohen_kappa_score(y, y_pred)
   
    print("---------------------------------------")
    print(dataType)
    print("  Accuracy = {}".format(accuracy))
    print("  F1 Score = {}".format(f1))
    print("  ROC Area under Curve = {}".format(roc_auc))
    print("  Cohen's Kappa = {}".format(coh_kap))
    print(classification_report(y,y_pred,digits=5))
        
    cm = confusion_matrix(y, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix - '+dataType)
    plt.show()

    probs = model.predict_proba(x)  
    print(probs)
    probs = probs[:, 1]  
    fper, tper, thresholds = roc_curve(y, probs) 
    plot_roc_cur(fper, tper)
    
    # Calculate precision, recall, and thresholds
    precision, recall, thresholds = precision_recall_curve(y, probs)
    

    # Plot Precision-Recall curve
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, marker='.', label=model[-1].__class__.__name__)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve\nF1 Score: {:.3f}'.format(f1))
    plt.grid(True)
    plt.legend()
    plt.show()

    rs = Result_Score()
    rs.model = model
    rs.accuracy = accuracy
    rs.roc_auc = roc_auc
    rs.coh_kap = coh_kap
    rs.f1 = f1
    rs.fper = fper
    rs.tper = tper
    rs.precision = precision
    rs.recall = recall

    return rs

In [ ]:
def plot_roc_cur(fper, tper):  
    plt.plot(fper, tper, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
def get_feature_importance(modelOrPipeline, x_train_columns):
    
    if hasattr(modelOrPipeline, 'steps'):
        model = modelOrPipeline[-1]
    else:
        model = modelOrPipeline

    importances_df = pd.DataFrame()
    if hasattr(model, "feature_importances_"):
        importances = model.feature_importances_

        # 確保 features 是一維純文字索引
        features = pd.Index(x_train_columns).astype(str)

        # 建立資料表
        importances_df = pd.DataFrame({'Feature': features, 'Importance': importances})
        importances_df = importances_df.sort_values(by='Importance', ascending=False)
        print(importances_df)
        # 畫圖
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=importances_df.head(20))
        plt.title('Top 20 Feature Importances')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.show()
    elif  hasattr(model, "coef_"):
        importances = model.coef_[0]
        features = pd.Index(x_train_columns).astype(str)
        importances_df = pd.DataFrame({'Feature': features, 'Importance': importances})
        importances_df = importances_df.reindex(importances_df['Importance'].abs().sort_values(ascending=False).index)
        print(importances_df)
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=importances_df.head(20))
        plt.title('Top 20 Feature Importances')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.show()
    return importances_df